In [1]:
import cv2
import dlib
import os
from shutil import copyfile
import numpy as np

def get_face_encoding(image_path):
    # Sử dụng dlib để lấy mã hóa khuôn mặt
    detector = dlib.get_frontal_face_detector()
    shape_predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    face_recognition_model = dlib.face_recognition_model_v1("dlib_face_recognition_resnet_model_v1.dat")

    img = cv2.imread(image_path)
    dets = detector(img, 1)

    if len(dets) > 0:
        shape = shape_predictor(img, dets[0])
        face_descriptor = face_recognition_model.compute_face_descriptor(img, shape)
        return face_descriptor
    else:
        return None

def classify_faces(source_folder, output_folder):
    # Lấy danh sách tất cả các file trong thư mục nguồn
    image_files = [f for f in os.listdir(source_folder) if os.path.isfile(os.path.join(source_folder, f))]

    # Dictionary để lưu trữ mã hóa khuôn mặt của từng người
    face_encodings = {}

    for image_file in image_files:
        image_path = os.path.join(source_folder, image_file)

        # Lấy mã hóa khuôn mặt từ hình ảnh
        face_encoding = get_face_encoding(image_path)

        if face_encoding:
            # Kiểm tra xem đã có folder cho người này chưa
            person_folder = None
            for person, encoding in face_encodings.items():
                # Convert face_encoding và encoding thành numpy arrays
                face_encoding_np = np.array(face_encoding)
                encoding_np = np.array(encoding)

                # So sánh mã hóa để xác định xem có thuộc về cùng một người hay không
                if cv2.norm(face_encoding_np, encoding_np, cv2.NORM_L2) < 0.4:
                    person_folder = person
                    break

            if not person_folder:
                # Nếu không có folder cho người này, tạo mới
                person_folder = os.path.join(output_folder, f"Person_{len(face_encodings) + 1}")
                os.makedirs(person_folder)
                face_encodings[person_folder] = face_encoding

            # Lưu hình ảnh vào folder tương ứng
            copyfile(image_path, os.path.join(person_folder, image_file))

if __name__ == "__main__":
    source_folder = "./cropped_train/"
    output_folder = "./similar_faces/"

    classify_faces(source_folder, output_folder)
